This script reformats NPD Well Headers for IC so that column names match up perfectly first time.<br>
Prepares well header files for nearly 1900 exploration and 5000 development wells in seconds.<br>
Automatically creates separate URL/Reference files for IC with links to NPD FactPages, FactMaps and Press Releases.<br>
Advises which Well Attibutes you need to manually create in IC.

<b>Imports NPD well headers from:</b><br>
- NPD Factpages > Wellbore > Table View > Exploration > All - Long List (<a href="http://factpages.npd.no/factpages/Default.aspx?culture=en&nav1=wellbore&nav2=TableView|Exploration|All">permanent link</a>)<br>
- NPD Factpages > Wellbore > Table View > Development > All - Long List. (<a href="http://factpages.npd.no/factpages/Default.aspx?culture=en&nav1=wellbore&nav2=TableView|Development">permanent link</a>)

<b>Run the script as it is, or edit options to:</b>
- A. Rename dozens of header fields to match IC defaults, e.g. 'Entered date' to 'SPUD date'.
- B. Truncate well list based on any field, e.g. 'Location: Barent's Sea'.
- C. Add/remove spaces or characters in well names.
- D. Concatenate cells to create 'Grid system' in IC format, e.g. "ED50 / UTM Zone 31N".
- E. Concatenate Status & Content to match IC's Well Symbols dictionary, e.g. "P & A Gas/Condensate".
- F. Add and populate new columns, e.g. 'Country: NORWAY'.
- G. Copy data from one column to another, e.g. 'NPDID wellbore' to 'UWI Number'.
- H. Re-order all columns, and delete unwanted columns.
- I. Convert three URL columns into rows to match format required for IC References.
- J. Create separate well header files for exploration and development wells, or combine into one.

<b>Outputs four files for import to IC:</b>
- IC_explo_references.csv
- IC_dev_references.csv
- IC_wellbore_exploration.CSV
- IC_wellbore_development.CSV

In [46]:
#to fix:
#fix norwegian character export.
#write function to advise which well attributes the user needs to create.
#Ask about script to create well attributes on the fly.

import pandas as pd
import numpy as np
from pandas import ExcelFile
from pandas import ExcelWriter

#Import the two Excel files and assign to dataframes
df_explo = pd.read_excel('wellbore_exploration_all.xls', sheet_name='wellbore_exploration_all')
df_dev = pd.read_excel('wellbore_development_all.xls', sheet_name='wellbore_development_all')

#Change Pandas display settings to show all columns
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

#Print the original column titles found in each dataframe.
print("Exploration well header column titles:")
print(list(df_explo.columns))
print("\n Development well header column titles:")
print(list(df_dev.columns))

WARNING *** file size (2924480) not 512 + multiple of sector size (512)
WARNING *** file size (6545344) not 512 + multiple of sector size (512)
Exploration well header column titles:
['Wellbore name', 'Well name', 'Drilling operator', 'Drilled in production licence', 'Purpose', 'Status', 'Content', 'Type', 'Subsea', 'Entered date', 'Completed date', 'Field', 'Drill permit', 'Discovery', 'Discovery wellbore', 'Bottom hole temperature [°C]', 'Seismic location', 'Maximum inclination [°]', 'Kelly bushing elevation [m]', 'Final vertical depth (TVD) [m RKB]', 'Total depth (MD) [m RKB]', 'Water depth [m]', 'Kick off  point [m RKB]', 'Oldest penetrated age', 'Oldest penetrated formation', 'Main area', 'Drilling facility', 'Drilling facility type', 'Drilling facility category', 'Licensing activity awarded in', 'Multilateral', 'Purpose - planned', 'Entry year', 'Completed year', 'Reclassified from/to wellbore', 'Reentry activity', 'Plot symbol', '1st level with HC, formation', '1st level with HC

In [3]:
#Print the number of rows and columns in each dataframe.
print('No. rows, columns in Exploration wells:', df_explo.shape)
print('No. rows, columns in Development wells:', df_dev.shape)

No. rows, columns in Exploration wells: (1898, 81)
No. rows, columns in Development wells: (5007, 71)


In [4]:
#Compare and contrast the well header fields in each dataframe.
explo_columns = df_explo.columns.tolist()
dev_columns = df_dev.columns.tolist()

print('Column headers unique to Exploration wells:')
print(list(set(explo_columns) - set(dev_columns)))
print(' ')
print('Column headers unique to Development wells:')
print(list(set(dev_columns) - set(explo_columns)))
print(' ')
print('Column headers shared by both Exploration and Development wells:')
print(set(explo_columns).intersection(dev_columns))

Column headers unique to Exploration wells:
['Reentry', 'Reentry activity', '2nd level with HC, formation', 'Drilling days', '1st level with HC, formation', '3rd level with HC, age', 'Maximum inclination [°]', 'Oldest penetrated age', 'Pressrelease url', 'Bottom hole temperature [°C]', 'Seismic location', 'Oldest penetrated formation', '3rd level with HC, formation', '2nd level with HC, age', '1st level with HC, age']
 
Column headers unique to Development wells:
['Production facility', 'Predrilled entry date', 'Content - planned', 'NPDID production facility', 'Predrilled completion date']
 
Column headers shared by both Exploration and Development wells:
{'EW UTM [m]', 'NPDID drilling facility', 'Date all updated', 'NS code', 'Discovery wellbore', 'Subsea', 'Wellbore name', 'Drilling facility type', 'Wellbore name, part 6', 'NS minutes', 'Licensing activity awarded in', 'Completed year', 'NS degrees', 'Discovery', 'Completed date', 'Entry year', 'Entered date', 'Kick off  point [m RKB

In [5]:
#Print top 5 rows in formatted dataframe
df_explo.head()

,Wellbore name,Well name,Drilling operator,Drilled in production licence,Purpose,Status,Content,Type,Subsea,Entered date,Completed date,Field,Drill permit,Discovery,Discovery wellbore,Bottom hole temperature [°C],Seismic location,Maximum inclination [°],Kelly bushing elevation [m],Final vertical depth (TVD) [m RKB],Total depth (MD) [m RKB],Water depth [m],Kick off point [m RKB],Oldest penetrated age,Oldest penetrated formation,Main area,Drilling facility,Drilling facility type,Drilling facility category,Licensing activity awarded in,Multilateral,Purpose - planned,Entry year,Completed year,Reclassified from/to wellbore,Reentry activity,Plot symbol,"1st level with HC, formation","1st level with HC, age","2nd level with HC, formation","2nd level with HC, age","3rd level with HC, formation","3rd level with HC, age",Drilling days,Reentry,Geodetic datum,NS degrees,NS minutes,NS seconds,NS code,EW degrees,EW minutes,EW seconds,EW code,NS decimal degrees,EW decimal degrees,NS UTM [m],EW UTM [m],UTM zone,"Wellbore name, part 1","Wellbore name, part 2","Wellbore name, part 3","Wellbore name, part 4","Wellbore name, part 5","Wellbore name, part 6",Pressrelease url,FactPage url,Factmaps,DISKOS Well Type,DISKOS Wellbore Parent,Publication date,Release date,NPDID wellbore,NPDID discovery,NPDID field,NPDID drilling facility,NPDID wellbore reclassified from,NPDID production licence drilled in,Date main level updated,Date all updated,Date sync NPD
0,1/2-1,1/2-1,Phillips Petroleum Norsk AS,143,WILDCAT,P&A,OIL,EXPLORATION,NO,1989-03-20,1989-06-04,BLANE,604-L,1/2-1 Blane,YES,147.0,PW 8303A - 10 SP. 290,2.0,22.0,0.0,3574.0,72.0,NaN,CAMPANIAN,TOR FM,NORTH SEA,ROSS ISLE,SEMISUB STEEL,MOVEABLE,12,NO,WILDCAT,1989,1989,NaN,NaN,5,FORTIES FM,PALEOCENE,NaN,NaN,,,77,NO,ED50,56,53,15.07,N,2,28,35.700001,E,56.887519,2.476583,6305128.26,468106.29,31,1,2,,1,NaN,NaN,,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=1382,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=1382,initial,,2007-12-19,1991-06-04,1382,43814.0,3437650.0,296245.0,0,21956,2019-04-02,2019-04-02,21.06.2019
1,1/2-2,1/2-2,Paladin Resources Norge AS,143 CS,WILDCAT,P&A,OIL SHOWS,EXPLORATION,NO,2005-12-14,2006-02-02,NaN,1103-L,NaN,NO,138.0,inline 7429-trace 4824 Survey PGS CGMNOR,4.9,40.0,3432.0,3434.0,74.0,NaN,PALEOCENE,EKOFISK FM,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,12,NO,WILDCAT,2005,2006,NaN,NaN,12,NaN,NaN,NaN,NaN,,,51,NO,ED50,56,59,32.00,N,2,29,47.660000,E,56.992222,2.496572,6316774.33,469410.10,31,1,2,,2,NaN,NaN,https://www.npd.no/fakta/nyheter/Resultat-av-leteboring/2006/1-2-2/,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=5192,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=5192,initial,,2008-08-15,2008-02-02,5192,NaN,NaN,278245.0,0,2424919,2019-04-09,2019-04-09,21.06.2019
2,1/3-1,1/3-1,A/S Norske Shell,011,WILDCAT,P&A,GAS,EXPLORATION,NO,1968-07-06,1968-11-11,NaN,15-L,1/3-1,YES,182.0,LINE 5651 SP. E165,18.0,26.0,0.0,4877.0,71.0,NaN,LATE PERMIAN,ZECHSTEIN GP,NORTH SEA,ORION,JACK-UP 3 LEGS,MOVEABLE,1-A,NO,WILDCAT,1968,1968,NaN,NaN,9,TOR FM,LATE CRETACEOUS,CROMER KNOLL GP,EARLY CRETACEOUS,,,129,NO,ED50,56,51,21.00,N,2,51,5.000000,E,56.855833,2.851389,6301488.86,490936.87,31,1,3,,1,NaN,NaN,,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=154,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=154,initial,,2010-04-30,1970-11-11,154,43820.0,NaN,288604.0,0,20844,2016-10-14,2019-03-15,21.06.2019
3,1/3-2,1/3-2,A/S Norske Shell,011,WILDCAT,P&A,DRY,EXPLORATION,NO,1969-05-14,1969-07-27,NaN,26-L,NaN,NO,143.0,LINE 5656 SP.5125,0.0,26.0,0.0,4297.0,73.0,NaN,EARLY CRETACEOUS,SOLA FM,NORTH SEA,SEDNETH I,SEMISUB STEEL,MOVEABLE,1-A,NO,WILDCAT,1969,1969,NaN,NaN,1,NaN,NaN,NaN,NaN,,,75,NO,ED50,56,56,10.00,N,2,45,0.000000,E,56.936111,2.750000,6310443.20,484786.28,31,1,3,,2,NaN,N

In [6]:
df_dev.head()

,Wellbore name,Well name,Drilling operator,Drilled in production licence,Status,Purpose,Purpose - planned,Content,Type,Subsea,Entered date,Completed date,Predrilled entry date,Predrilled completion date,Field,Drill permit,Discovery,Discovery wellbore,Kelly bushing elevation [m],Final vertical depth (TVD) [m RKB],Total depth (MD) [m RKB],Water depth [m],Kick off point [m RKB],Main area,Drilling facility,Drilling facility type,Drilling facility category,Production facility,Licensing activity awarded in,Multilateral,Content - planned,Entry year,Completed year,Reclassified from/to wellbore,Plot symbol,Geodetic datum,NS degrees,NS minutes,NS seconds,NS code,EW degrees,EW minutes,EW seconds,EW code,NS decimal degrees,EW decimal degrees,NS UTM [m],EW UTM [m],UTM zone,"Wellbore name, part 1","Wellbore name, part 2","Wellbore name, part 3","Wellbore name, part 4","Wellbore name, part 5","Wellbore name, part 6",FactPage url,Factmaps,DISKOS Well Type,DISKOS Wellbore Parent,NPDID wellbore,NPDID discovery,NPDID field,Publication date,Release date,NPDID production licence drilled in,NPDID drilling facility,NPDID production facility,NPDID wellbore reclassified from,Date main level updated,Date all updated,Date sync NPD
0,1/3-A-1 H,1/3-A-1,DONG E&P Norge AS,274,CLOSED,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2011-07-22,2011-09-21,NaT,NaT,OSELVAR,3365-P,1/3-6 Oselvar,NO,45.0,3163.0,5927.0,72.0,NaN,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,OSELVAR,NST2001,NO,OIL,2011,2011,NaN,50,ED50,56,55,55.060001,N,2,40,16.66,E,56.931961,2.671294,6310001.50,479994.47,31,1,3,A,1,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=6612,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=6612,initial,,6612,43832,5506919.0,NaT,2013-09-21,2060266,278245.0,410592.0,0,2019-03-23,2019-03-23,24.06.2019
1,1/3-A-2 H,1/3-A-2,DONG E&P Norge AS,274,CLOSED,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2011-11-18,2012-01-19,2011-06-19,2011-07-04,OSELVAR,3366-P,1/3-6 Oselvar,NO,45.0,3170.0,5882.0,72.0,NaN,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,OSELVAR,NST2001,NO,OIL,2011,2012,NaN,50,ED50,56,55,54.889999,N,2,40,16.67,E,56.931914,2.671297,6309996.24,479994.61,31,1,3,A,2,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=6613,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=6613,initial,,6613,43832,5506919.0,NaT,2014-01-19,2060266,278245.0,410592.0,0,2019-03-23,2019-03-23,24.06.2019
2,1/3-A-3 H,1/3-A-3,DONG E&P Norge AS,274,CLOSED,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2012-03-04,2012-05-14,2011-07-05,2011-07-21,OSELVAR,3367-P,1/3-6 Oselvar,NO,45.0,3171.0,6665.0,72.0,NaN,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,OSELVAR,NST2001,NO,OIL,2012,2012,NaN,50,ED50,56,55,55.070000,N,2,40,17.32,E,56.931964,2.671478,6310001.76,480005.63,31,1,3,A,3,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=6614,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=6614,initial,,6614,43832,5506919.0,NaT,2014-05-14,2060266,278245.0,410592.0,0,2019-03-23,2019-03-23,24.06.2019
3,1/3-K-1,1/3-K-1,BP Amoco Norge AS,065,PRODUCING,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,NO,1998-05-08,1998-07-31,NaT,NaT,TAMBAR,1942-P,1/3-3 Tambar,NO,50.0,4386.0,4543.0,68.0,NaN,NORTH SEA,TRANSOCEAN NORDIC,JACK-UP 3 LEGS,MOVEABLE,TAMBAR,6,NO,OIL,1998,1998,1/3-9 S,50,ED50,56,58,57.970001,N,2,57,31.33,E,56.982769,2.958703,6315610.11,497490.00,31,1,3,K,1,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=4320,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=4320,initial,,4320,43826,1028599.0,NaT,2000-07-31,21316,296191.0,280793.0,3362,2019-03-26,2019-03-26,24.06.2019
4,1/3-K-2,1/3-K-2,Aker BP ASA,065,PLUGGED,OBSERVATION,PRODUCTION,NOT APPLICABLE,DEVELOPMENT,NO,2018-01-02,2018-01-19,2017-10-27,2017-1

In [7]:
df_explo.tail()

,Wellbore name,Well name,Drilling operator,Drilled in production licence,Purpose,Status,Content,Type,Subsea,Entered date,Completed date,Field,Drill permit,Discovery,Discovery wellbore,Bottom hole temperature [°C],Seismic location,Maximum inclination [°],Kelly bushing elevation [m],Final vertical depth (TVD) [m RKB],Total depth (MD) [m RKB],Water depth [m],Kick off point [m RKB],Oldest penetrated age,Oldest penetrated formation,Main area,Drilling facility,Drilling facility type,Drilling facility category,Licensing activity awarded in,Multilateral,Purpose - planned,Entry year,Completed year,Reclassified from/to wellbore,Reentry activity,Plot symbol,"1st level with HC, formation","1st level with HC, age","2nd level with HC, formation","2nd level with HC, age","3rd level with HC, formation","3rd level with HC, age",Drilling days,Reentry,Geodetic datum,NS degrees,NS minutes,NS seconds,NS code,EW degrees,EW minutes,EW seconds,EW code,NS decimal degrees,EW decimal degrees,NS UTM [m],EW UTM [m],UTM zone,"Wellbore name, part 1","Wellbore name, part 2","Wellbore name, part 3","Wellbore name, part 4","Wellbore name, part 5","Wellbore name, part 6",Pressrelease url,FactPage url,Factmaps,DISKOS Well Type,DISKOS Wellbore Parent,Publication date,Release date,NPDID wellbore,NPDID discovery,NPDID field,NPDID drilling facility,NPDID wellbore reclassified from,NPDID production licence drilled in,Date main level updated,Date all updated,Date sync NPD
1893,7324/10-1,7324/10-1,Den norske stats oljeselskap a.s,162,WILDCAT,P&A,SHOWS,EXPLORATION,NO,1989-06-03,1989-08-19,NaN,609-L,NaN,NO,119.0,GLR 3-87-19B SP. 3290,2.8,23.0,2919.0,2919.0,408.0,NaN,EARLY TRIASSIC,HAVERT FM,BARENTS SEA,ROSS RIG (2),SEMISUB STEEL,MOVEABLE,12-B,NO,WILDCAT,1989,1989,NaN,NaN,12,NaN,NaN,NaN,NaN,,,78,NO,ED50,73,9,49.450001,N,24,18,47.619999,E,73.163736,24.313228,8120918.38,413162.08,35,7324,10,,1,NaN,NaN,,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=1411,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=1411,initial,,2005-01-11,1991-08-19,1411,NaN,NaN,294382.0,0,22108,2016-05-20,2019-03-15,21.06.2019
1894,7325/1-1,7325/1-1,Statoil Petroleum AS,615,WILDCAT,P&A,GAS,EXPLORATION,NO,2014-06-23,2014-07-21,NaN,1526-L,7325/1-1 (Atlantis),YES,110.0,3D survey MC3D-HFC09-inline 2215 & xline 8697,2.7,40.0,2865.0,2865.0,487.0,NaN,EARLY TRIASSIC,HAVERT FM,BARENTS SEA,TRANSOCEAN SPITSBERGEN,SEMISUB STEEL,MOVEABLE,21,NO,WILDCAT,2014,2014,NaN,NaN,9,SNADD FM,LATE TRIASSIC,NaN,NaN,,,29,NO,ED50,73,54,48.700001,N,25,7,0.170000,E,73.913528,25.116714,8203538.99,441758.31,35,7325,1,,1,NaN,NaN,https://www.npd.no/fakta/nyheter/Resultat-av-leteboring/2014/73251-1/,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=7501,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=7501,initial,,2016-07-21,2016-07-21,7501,24689817.0,NaN,404201.0,0,20429544,2019-04-09,2019-04-09,21.06.2019
1895,7325/4-1,7325/4-1,Statoil Petroleum AS,855,WILDCAT,P&A,OIL/GAS,EXPLORATION,NO,2017-07-19,2017-08-03,NaN,1661-L,7325/4-1 (Gemini Nord),YES,NaN,NaN,NaN,32.0,NaN,1210.0,447.0,NaN,LATE TRIASSIC,NaN,BARENTS SEA,SONGA ENABLER,SEMISUB STEEL,MOVEABLE,23,NO,WILDCAT,2017,2017,NaN,NaN,13,NaN,MIDDLE JURASSIC,NaN,LATE TRIASSIC,NaN,,16,NO,ED50,73,38,57.549999,N,25,10,41.740002,E,73.649319,25.178261,8174015.52,442761.29,35,7325,4,,1,NaN,NaN,https://www.npd.no/fakta/nyheter/Resultat-av-leteboring/2017/73254-1/,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=8211,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=8211,initial,,2019-08-03,2019-08-03,8211,29469097.0,NaN,439972.0,0,28168828,2019-04-06,2019-04-06,21.06.2019
1896,7335/3-1,7335/3-1,Equinor Energy AS,859,WILDCAT,P&A,DRY,EXPLORATION,NO,2019-05-13,2019-06-15,NaN,1760-L,NaN,NO,NaN,ST14005T15 Inline: 4481. Xline: 8373,NaN,31.0,NaN,4300.0,239.0,NaN,TRIASSIC,NaN,BAREN

In [8]:
df_dev.tail()

,Wellbore name,Well name,Drilling operator,Drilled in production licence,Status,Purpose,Purpose - planned,Content,Type,Subsea,Entered date,Completed date,Predrilled entry date,Predrilled completion date,Field,Drill permit,Discovery,Discovery wellbore,Kelly bushing elevation [m],Final vertical depth (TVD) [m RKB],Total depth (MD) [m RKB],Water depth [m],Kick off point [m RKB],Main area,Drilling facility,Drilling facility type,Drilling facility category,Production facility,Licensing activity awarded in,Multilateral,Content - planned,Entry year,Completed year,Reclassified from/to wellbore,Plot symbol,Geodetic datum,NS degrees,NS minutes,NS seconds,NS code,EW degrees,EW minutes,EW seconds,EW code,NS decimal degrees,EW decimal degrees,NS UTM [m],EW UTM [m],UTM zone,"Wellbore name, part 1","Wellbore name, part 2","Wellbore name, part 3","Wellbore name, part 4","Wellbore name, part 5","Wellbore name, part 6",FactPage url,Factmaps,DISKOS Well Type,DISKOS Wellbore Parent,NPDID wellbore,NPDID discovery,NPDID field,Publication date,Release date,NPDID production licence drilled in,NPDID drilling facility,NPDID production facility,NPDID wellbore reclassified from,Date main level updated,Date all updated,Date sync NPD
5002,7122/10-E-2 H,7122/10-E-2,Eni Norge AS,229,PRODUCING,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2015-02-16,2015-05-09,2015-02-07,2015-02-09,GOLIAT,3840-P,7122/7-1 Goliat,NO,34.0,1175.0,3024.0,323.0,NaN,BARENTS SEA,SCARABEO 8,SEMISUB STEEL,MOVEABLE,GOLIAT E,BARENTSHAVPROSJEKTET-97,NO,OIL,2015,2015,NaN,50,ED50,71,14,36.209999,N,22,14,59.84,E,71.243392,22.249956,7905219.28,544858.15,34,7122,10,E,2,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=7643,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=7643,initial,,7643,1340775,5774394.0,NaT,2017-05-09,22644,404415.0,411122.0,0,2019-03-22,2019-03-22,24.06.2019
5003,7122/10-E-3 H,7122/10-E-3,Eni Norge AS,229,PRODUCING,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2015-02-18,2015-03-29,2015-02-13,2015-02-13,GOLIAT,3841-P,7122/7-1 Goliat,NO,34.0,1862.0,3800.0,323.0,NaN,BARENTS SEA,SCARABEO 8,SEMISUB STEEL,MOVEABLE,GOLIAT E,BARENTSHAVPROSJEKTET-97,NO,OIL,2015,2015,NaN,50,ED50,71,14,36.340000,N,22,15,0.53,E,71.243428,22.250147,7905223.45,544864.94,34,7122,10,E,3,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=7644,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=7644,initial,,7644,1340775,5774394.0,NaT,2017-03-29,22644,404415.0,411122.0,0,2019-03-22,2019-03-22,24.06.2019
5004,7122/10-E-4 H,7122/10-E-4,Eni Norge AS,229,PRODUCING,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2018-06-09,2018-07-07,NaT,NaT,GOLIAT,4672-P,7122/7-1 Goliat,NO,32.0,1858.0,4413.0,323.0,NaN,BARENTS SEA,SONGA ENABLER,SEMISUB STEEL,MOVEABLE,GOLIAT E,BARENTSHAVPROSJEKTET-97,NO,OIL,2018,2018,NaN,50,ED50,71,14,35.930000,N,22,15,1.23,E,71.243314,22.250342,7905210.89,544872.18,34,7122,10,E,4,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=8440,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=8440,initial,,8440,1340775,5774394.0,NaT,2020-07-07,22644,439972.0,411122.0,0,2019-03-22,2019-03-22,24.06.2019
5005,7122/10-I-2 H,7122/10-I-2,Eni Norge AS,229,INJECTING,INJECTION,INJECTION,GAS,DEVELOPMENT,YES,2014-02-08,2014-03-16,NaT,NaT,GOLIAT,3710-P,7122/7-1 Goliat,NO,34.0,2116.0,2510.0,358.0,NaN,BARENTS SEA,SCARABEO 8,SEMISUB STEEL,MOVEABLE,GOLIAT I,BARENTSHAVPROSJEKTET-97,NO,GAS,2014,2014,NaN,91,ED50,71,14,32.950001,N,22,15,3.55,E,71.242486,22.250986,7905119.06,544897.22,34,7122,10,I,2,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=7402,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=7402,initial,,7402,1340775,5774394.0,NaT,2016-03-16,22644,404415.0,411354.0,0,2019-04-02,2019-04-02,24.06.2019
50

<br><b>SECTION A. Rename columns to match IC defaults</b>

In [1]:
#For ease, the order shown here matches our final output.

columns_to_rename = {
    'Wellbore name' : 'Well',    
    'Well name' : 'Alternate 1',
    'Drilling operator' : 'Operator',
    'Drilled in production licence' : 'Licence number',
    'Purpose' : 'Intent',
    'Purpose - planned' : 'Intent - planned',
    'Status' : 'Well status',
    'Content' : 'Well content',
    #Well status & Well content will be combined into a new column for IC called Status, e.g. "P & A Oil & Gas Condensate".
    'Entered date' : 'SPUD date',
    'Completed date' : 'Completion date',
    'Discovery' : 'Discovery name',
    'Seismic location' : 'Seismic line',
    'Kelly bushing elevation [m]' : 'KBE',
    'Total depth (MD) [m RKB]' : 'Terminal depth',
    'Water depth [m]' : 'Water depth',
    'Main area' : 'Location',
    'Drilling facility' : 'Facility',
    '1st level with HC, formation' : '1st level with HC formation',
    '1st level with HC, age' : '1st level with HC age',
    '2nd level with HC, formation' : '2nd level with HC formation',
    '2nd level with HC, age' : '2nd level with HC age',
    '3rd level with HC, formation' : '3rd level with HC formation',
    '3rd level with HC, age' : '3rd level with HC age',
    'Geodetic datum' : 'Geodatum',
    'NS decimal degrees' : 'Latitude',
    'EW decimal degrees' : 'Longitude',
    'NS UTM [m]' : 'Surface Y',
    'EW UTM [m]' : 'Surface X',
    'Wellbore name, part 1' : 'Wellbore name part 1',
    'Wellbore name, part 2' : 'Wellbore name part 2', 
    'Wellbore name, part 3' : 'Wellbore name part 3',
    'Wellbore name, part 4' : 'Wellbore name part 4',
    'Wellbore name, part 5' : 'Wellbore name part 5',
    'Wellbore name, part 6' : 'Wellbore name part 6',
    'Pressrelease url' : 'Press Release URL',
    'FactPage url' : 'FactPage URL',
    'Factmaps' : 'FactMaps URL'
          }

#Apply the new names to each of the dataframes
df_explo.rename(columns=columns_to_rename, inplace=True)
df_dev.rename(columns=columns_to_rename, inplace=True)

#Duplicate a couple of columns, where we want to keep the original
df_explo['Quadrant'] = df_explo['Wellbore name part 1']
df_dev['Quadrant'] = df_dev['Wellbore name part 1']
df_explo['Block'] = df_explo['Wellbore name part 2']
df_dev['Block'] = df_dev['Wellbore name part 2']
df_explo['UWI number'] = df_explo['NPDID wellbore']
df_dev['UWI number'] = df_dev['NPDID wellbore']

NameError: name 'df_explo' is not defined

In [10]:
#List of IC's default well header fields. Any others will need to be created manually under Wells > Attributes.

#You need to manually create the following 42 entries under Wells > Attributes.
#If you choose to rename any of these, ensure you add to rename section above.

#Or using Database ribbon > Database Utilities > Scripts tab??????????????????????????
#Script Type: Database Scripts. Create New.

# Intent - planned
# Well status
# Well content
# Content - planned #DEV WELLS ONLY
# Type
# Subsea
# Predrilled entry date #DEV WELLS ONLY
# Predrilled completion date #DEV WELLS ONLY
# Drill permit
# Discovery wellbore
# Bottom hole temperature [°C]
# Maximum inclination [°]
# Final vertical depth (TVD) [m RKB]
# Kick off point [m RKB]
# Oldest penetrated age
# Oldest penetrated formation
# Drilling facility type
# Drilling facility category
# Licensing activity awarded in
# Production facility #DEV WELLS ONLY
# Multilateral
# Entry year
# Completed year
# Reclassified from/to wellbore
# Reentry activity
# Plot symbol
# 1st level with HC formation
# 1st level with HC age
# 2nd level with HC formation
# 2nd level with HC age
# 3rd level with HC formation
# 3rd level with HC age
# Drilling days
# Reentry
# DISKOS Well Type
# DISKOS Wellbore Parent
# Publication date
# Release date
# NPDID wellbore
# NPDID discovery
# NPDID field
# NPDID production facility #DEV WELLS ONLY
# NPDID drilling facility
# NPDID wellbore reclassified from
# NPDID production licence drilled in
# Date main level updated
# Date all updated
# Date sync NPD

In [11]:
#Option to add/remove all spaces/characters from well names or any other column.
#df_explo['Well'] = df_explo['Well'].str.replace(' ', '')
#df_explo['Well'].head(n=20)

In [22]:
#Truncate well list based on a particular column and value.
# ONLY RUN THIS CELL IF YOU WANT TO CROP THE FILE. ALL OUTPUT FILES ARE AFFECTED.
#This example lets you crop well list based on Location: NORTH SEA, NORWEGIAN SEA, BARENTS SEA.
#For multiple values, replace 'BARENTS SEA' with ['BARENTS SEA', 'NORWEGIAN SEA']?????????????????????????
#Print the number of 

#Select the column and value you want to filter on
fltr_column = 'Location'
fltr_value = 'BARENTS SEA'

#Apply the filter to the dataframes
indexNames = df_explo[df_explo[fltr_column]!=fltr_value].index
df_explo.drop(indexNames , inplace=True)
indexNames = df_dev[df_dev[fltr_column]!=fltr_value].index
df_dev.drop(indexNames , inplace=True)

#Get dataframe shapse and unpack tuples
explo_shp = df_explo.shape
(exploRows, exploCols) = explo_shp
df_dev_shp = df_dev.shape
(devRows, devCols) = explo_shp

#Print out the results
print('After filtering on', fltr_column, ':', fltr_value,
      'you are left with', exploRows, 'rows and', exploCols, 'columns', 'for Exploration wells, and', 
      devRows, 'rows and', devCols, 'columns', 'for Development wells.')

After filtering on Location : BARENTS SEA you are left with 163 rows and 84 columns for Exploration wells, and 163 rows and 84 columns for Development wells.


In [23]:
df_explo.head()

,Well,Alternate 1,Operator,Licence number,Intent,Well status,Well content,Type,Subsea,SPUD date,Completion date,Field,Drill permit,Discovery name,Discovery wellbore,Bottom hole temperature [°C],Seismic line,Maximum inclination [°],KBE,Final vertical depth (TVD) [m RKB],Terminal depth,Water depth,Kick off point [m RKB],Oldest penetrated age,Oldest penetrated formation,Location,Facility,Drilling facility type,Drilling facility category,Licensing activity awarded in,Multilateral,Intent - planned,Entry year,Completed year,Reclassified from/to wellbore,Reentry activity,Plot symbol,1st level with HC formation,1st level with HC age,2nd level with HC formation,2nd level with HC age,3rd level with HC formation,3rd level with HC age,Drilling days,Reentry,Geodatum,NS degrees,NS minutes,NS seconds,NS code,EW degrees,EW minutes,EW seconds,EW code,Latitude,Longitude,Surface Y,Surface X,UTM zone,Wellbore name part 1,Wellbore name part 2,Wellbore name part 3,Wellbore name part 4,Wellbore name part 5,Wellbore name part 6,Press Release URL,FactPage URL,FactMaps URL,DISKOS Well Type,DISKOS Wellbore Parent,Publication date,Release date,NPDID wellbore,NPDID discovery,NPDID field,NPDID drilling facility,NPDID wellbore reclassified from,NPDID production licence drilled in,Date main level updated,Date all updated,Date sync NPD,Quadrant,Block,UWI number
1735,7016/2-1,7016/2-1,Eni Norge AS,529,WILDCAT,P&A,DRY,EXPLORATION,NO,2013-07-14,2013-11-03,NaN,1465-L,NaN,NO,0.0,Survey MCD 3D troms 111 inline 1408 & trace 3478,9.1,34.0,4060.0,4061.0,1366.0,NaN,LATE PALEOCENE,SOTBAKKEN GP,BARENTS SEA,SCARABEO 8,SEMISUB STEEL,MOVEABLE,20,NO,WILDCAT,2013,2013,NaN,NaN,1,NaN,NaN,NaN,NaN,,,113,NO,ED50,70,50,48.930000,N,16,33,33.230000,E,70.846925,16.559231,7861269.63,557093.48,33,7016,2,,1,NaN,NaN,https://www.npd.no/fakta/nyheter/resultat-av-leteboring/2013/70162-1/,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=7232,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=7232,initial,,2015-10-01,2015-10-01,7232,NaN,NaN,404415.0,0,5463559,2019-04-09,2019-04-09,21.06.2019,7016,2,7232
1736,7019/1-1,7019/1-1,Norsk Agip AS,201,WILDCAT,P&A,GAS,EXPLORATION,NO,2000-10-06,2000-12-03,NaN,984-L,7019/1-1,YES,108.0,94 TRM3D inline 1026 & crossline 1050,7.9,24.0,2998.0,3003.0,190.0,NaN,EARLY JURASSIC,TUBÅEN FM,BARENTS SEA,TRANSOCEAN ARCTIC,SEMISUB STEEL,MOVEABLE,14,NO,WILDCAT,2000,2000,NaN,NaN,9,STØ FM,MIDDLE JURASSIC,NaN,NaN,,,59,NO,ED50,70,55,5.100000,N,19,4,22.440001,E,70.918083,19.072900,7869589.80,429692.26,34,7019,1,,1,NaN,NaN,,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=4145,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=4145,initial,,2003-02-11,2002-12-03,4145,1340799.0,NaN,294382.0,0,22420,2019-03-22,2019-03-22,21.06.2019,7019,1,4145
1737,7117/9-1,7117/9-1,Norsk Hydro Produksjon AS,063,WILDCAT,P&A,DRY,EXPLORATION,NO,1982-04-20,1982-07-16,NaN,323-L,NaN,NO,112.0,79409 SP 128,7.5,23.0,3196.0,3200.0,261.0,NaN,EARLY CRETACEOUS,KOLMULE FM,BARENTS SEA,TREASURE SCOUT,SEMISUB STEEL,MOVEABLE,5,NO,WILDCAT,1982,1982,NaN,NaN,1,NaN,NaN,NaN,NaN,,,88,NO,ED50,71,22,51.049999,N,17,56,5.760000,E,71.380847,17.934933,7922625.97,604554.55,33,7117,9,,1,NaN,NaN,,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=49,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=49,initial,,2004-12-17,1984-07-16,49,NaN,NaN,287848.0,0,21300,2016-05-20,2019-03-15,21.06.2019,7117,9,49
1738,7117/9-2,7117/9-2,Norsk Hydro Produksjon AS,063,WILDCAT,P&A,DRY,EXPLORATION,NO,1983-05-07,1983-09-09,NaN,372-L,NaN,NO,173.0,X LINE 8103-819 SP:205+8103-610 SP:403,7.7,23.0,4991.0,5000.0,271.0,NaN,EARLY CRETACEOUS,KOLMULE FM,BARENTS SEA,TREASURE SCOUT,SEMISUB STEEL,MOVEABLE,5,NO,WILDCAT,1983,1983,NaN,NaN,1,NaN,NaN,NaN,NaN,,,126,NO,ED50,71,25,44.750000,N,17,53,32.450001,E,71.429097,17.892347,7927929.19,602781

In [24]:
#Creates 'Reference' files containing Press Release URL, FactPage URL and FactMaps URl for Exploration wells.
#Converts three columns into three rows. Adds a Title column and sorts by Well and Title.

df_explo_references = df_explo[['Well', 'Press Release URL', 'FactPage URL', 'FactMaps URL']]
df_explo_references = pd.melt(df_explo_references, id_vars='Well', value_vars=['Press Release URL', 'FactPage URL', 'FactMaps URL'], var_name='Title', value_name='URL')
df_explo_references.sort_values(['Well', 'Title'], inplace=True)
df_explo_references.to_csv('IC_explo_references.csv', index=False)

#Print the result
df_explo_references.head()

,Well,Title,URL
326,7016/2-1,FactMaps URL,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=7232
163,7016/2-1,FactPage URL,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=7232
0,7016/2-1,Press Release URL,https://www.npd.no/fakta/nyheter/resultat-av-leteboring/2013/70162-1/
327,7019/1-1,FactMaps URL,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=4145
164,7019/1-1,FactPage URL,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=4145


In [25]:
#As above, but creates 'Reference' file for Development Wells (minus the Press Release URL)
df_dev_references = df_dev[['Well', 'FactPage URL', 'FactMaps URL']]
df_dev_references = pd.melt(df_dev_references, id_vars='Well', value_vars=['FactPage URL', 'FactMaps URL'], var_name='Title', value_name='URL')
df_dev_references.sort_values(['Well', 'Title'], inplace=True)
df_dev_references.to_csv('IC_dev_references.csv', index=False)
df_dev_references.head()

,Well,Title,URL
45,7121/4-D-1 H,FactMaps URL,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=5034
0,7121/4-D-1 H,FactPage URL,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=5034
46,7121/4-D-2 H,FactMaps URL,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=5035
1,7121/4-D-2 H,FactPage URL,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=5035
47,7121/4-D-4 H,FactMaps URL,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=5036


In [27]:
#Add new column(s) and assign constant value, e.g. Country: NORWAY.
df_explo['Country'] = 'NORWAY'
df_dev['Country'] = 'NORWAY'
df_explo['Country'].head(n=2)

1735    NORWAY
1736    NORWAY
Name: Country, dtype: object

In [29]:
#Forces 'NPDID drilling facility' whole number, removing decimal place.
df_explo['NPDID drilling facility'].astype(int).head()
df_dev['NPDID drilling facility'].astype(int).head()

4962    289333
4963    289333
4964    289333
4965    289333
4966    289333
Name: NPDID drilling facility, dtype: int64

In [30]:
#Combine 'Well Status' and 'Well content' to create a new column called 'Status'.
#This links to and matches IC's Well Symbols graphic dictionary entries, e.g. "P&A Oil Shows"
#Changes 'P&A' to 'P & A'.
#First letter of each word capitalised to match the Well Symbols graphic dictionary in IC.

df_explo['Well status'] = df_explo['Well status'].replace(to_replace='P&A', value='P & A')
df_explo['Status'] = df_explo['Well status'] + ' ' + df_explo['Well content'].str.title()
df_explo['Status'].head(n=10)

1735    P & A Dry           
1736    P & A Gas           
1737    P & A Dry           
1738    P & A Dry           
1739    P & A Dry           
1740    P & A Dry           
1741    P & A Oil Shows     
1742    P & A Shows         
1743    P & A Gas/Condensate
1744    P & A Dry           
Name: Status, dtype: object

In [31]:
#Same as above but for Development wells
df_dev['Well status'] = df_dev['Well status'].replace(to_replace='P&A', value='P & A')
df_dev['Status'] = df_dev['Well status'] + ' ' + df_dev['Well content'].str.title()
df_dev['Status'].head(n=10)

4962    PRODUCING Gas        
4963    PRODUCING Gas        
4964    PRODUCING Gas        
4965    PRODUCING Gas        
4966    PRODUCING Gas        
4967    PRODUCING Gas        
4968    JUNKED Not Applicable
4969    INJECTING Co2        
4970    PRODUCING Gas        
4971    CLOSED Gas           
Name: Status, dtype: object

In [17]:
#Lists all unique entries under Well status, Well content and Status (the two combined) for both Explo and Dev wells.
#Use this to ensure you have dictionary entries for each, under Database > Graphic Dictionaries > Well Symbols.
#Either manipulate Status values here, or rename Graphic Dictionary entries in IC.

print('EXPLORATION WELLS - UNIQUE STATUS VALUES ---')
print('')
lst_explo_wellstatus = sorted(set(df_explo['Well status'].astype(str)))
print('Well status:', ', '.join(lst_explo_wellstatus))
print('')

lst_explo_wellcontent = sorted(set(df_explo['Well content'].astype(str)))
print('Well content:', ', '.join(lst_explo_wellcontent))
print('')

lst_explo_status = sorted(set(df_explo['Status'].astype(str)))
print('Status:', ', '.join(lst_explo_status))

print('')
print('')
print('DEVELOPMENT WELLS - UNIQUE STATUS VALUES ---')
print('')
lst_dev_wellstatus = sorted(set(df_dev['Well status'].astype(str)))
print('Well status:', ', '.join(lst_dev_wellstatus))
print('')

lst_dev_wellcontent = sorted(set(df_dev['Well content'].astype(str)))
print('Well content:', ', '.join(lst_dev_wellcontent))
print('')

lst_dev_status = sorted(set(df_dev['Status'].astype(str)))
print('Status:', ', '.join(lst_dev_status))

EXPLORATION WELLS - UNIQUE STATUS VALUES ---

Well status: DRILLING, JUNKED, P & A, SUSPENDED, WILL NEVER BE DRILLED

Well content: DRY, GAS, GAS SHOWS, GAS/CONDENSATE, NOT APPLICABLE, OIL, OIL SHOWS, OIL/GAS, OIL/GAS SHOWS, SHOWS, nan

Status: JUNKED Not Applicable, P & A Dry, P & A Gas, P & A Gas Shows, P & A Gas/Condensate, P & A Oil, P & A Oil Shows, P & A Oil/Gas, P & A Oil/Gas Shows, P & A Shows, SUSPENDED Dry, SUSPENDED Oil/Gas, SUSPENDED Oil/Gas Shows, SUSPENDED Shows, nan


DEVELOPMENT WELLS - UNIQUE STATUS VALUES ---

Well status: CLOSED, INJECTING, JUNKED, PLUGGED, PRODUCING

Well content: CO2, GAS, NOT APPLICABLE, OIL, WATER

Status: CLOSED Gas, INJECTING Co2, INJECTING Gas, INJECTING Water, JUNKED Not Applicable, PLUGGED Not Applicable, PLUGGED Oil, PLUGGED Water, PRODUCING Gas, PRODUCING Oil


In [32]:
#Concatenates cells to create a new columns 'Grid system' in IC format (e.g. "ED50 / UTM Zone 31N").
df_explo['Grid system'] = df_explo['Geodatum'] + '/ ' + 'UTM zone ' + df_explo['UTM zone'].map(str) + df_explo['NS code']
#Remove trailing space on Geodatum column (e.g. 'ED50 ' to 'ED50').
df_explo['Grid system'] = df_explo['Grid system'].str.replace('ED50 ', 'ED50')

#Same as above but for Development wells
df_dev['Grid system'] = df_dev['Geodatum'] + '/ ' + 'UTM zone ' + df_dev['UTM zone'].map(str) + df_dev['NS code']
df_dev['Grid system'] = df_dev['Grid system'].str.replace('ED50 ', 'ED50')
df_dev['Grid system'].head()

4962    ED50 / UTM zone 34N
4963    ED50 / UTM zone 34N
4964    ED50 / UTM zone 34N
4965    ED50 / UTM zone 34N
4966    ED50 / UTM zone 34N
Name: Grid system, dtype: object

In [33]:
#Prints out indices for Exploration and Development wells after all the above editing.
#Use these lists to check the current order of your columns in each, and consider how you might like to re-order them.

print(df_explo.columns)
print(df_dev.columns)

Index(['Well', 'Alternate 1', 'Operator', 'Licence number', 'Intent',
       'Well status', 'Well content', 'Type', 'Subsea', 'SPUD date',
       'Completion date', 'Field', 'Drill permit', 'Discovery name',
       'Discovery wellbore', 'Bottom hole temperature [°C]', 'Seismic line',
       'Maximum inclination [°]', 'KBE', 'Final vertical depth (TVD) [m RKB]',
       'Terminal depth', 'Water depth', 'Kick off  point [m RKB]',
       'Oldest penetrated age', 'Oldest penetrated formation', 'Location',
       'Facility', 'Drilling facility type', 'Drilling facility category',
       'Licensing activity awarded in', 'Multilateral', 'Intent - planned',
       'Entry year', 'Completed year', 'Reclassified from/to wellbore',
       'Reentry activity', 'Plot symbol', '1st level with HC formation',
       '1st level with HC age', '2nd level with HC formation',
       '2nd level with HC age', '3rd level with HC formation',
       '3rd level with HC age', 'Drilling days', 'Reentry', 'Geodatum',


In [34]:
#Specifies the order of columns for Exploration and Development wells in the final outputs.

exploColumnOrder = ['Well', 'Alternate 1', 'UWI number', 'Quadrant', 'Block', 'Operator', 'Licence number', 'Intent', 'Intent - planned', 'Well status', 'Well content', 
                    'Status', 'Type', 'Subsea', 'SPUD date', 'Completion date', 'Field', 'Drill permit', 'Discovery name', 
                    'Discovery wellbore', 'Bottom hole temperature [°C]', 'Seismic line', 'Maximum inclination [°]', 'KBE', 
                    'Final vertical depth (TVD) [m RKB]', 'Terminal depth', 'Water depth', 
                    'Kick off  point [m RKB]', 'Oldest penetrated age', 'Oldest penetrated formation', 'Location', 'Country', 'Facility', 
                    'Drilling facility type', 'Drilling facility category', 'Licensing activity awarded in', 
                    'Multilateral', 'Entry year', 'Completed year', 'Reclassified from/to wellbore', 'Reentry activity', 'Plot symbol', 
                    '1st level with HC formation', '1st level with HC age', '2nd level with HC formation', '2nd level with HC age', 
                    '3rd level with HC formation', '3rd level with HC age', 'Drilling days', 'Reentry', 'Geodatum', 'NS degrees', 
                    'NS minutes', 'NS seconds', 'NS code', 'EW degrees', 'EW minutes', 'EW seconds', 'EW code', 'Latitude', 'Longitude', 
                    'Surface X', 'Surface Y', 'UTM zone', 'Grid system', 'Wellbore name part 1', 'Wellbore name part 2', 'Wellbore name part 3', 
                    'Wellbore name part 4', 'Wellbore name part 5', 'Wellbore name part 6', 'Press Release URL', 'FactPage URL', 'FactMaps URL', 
                    'DISKOS Well Type', 'DISKOS Wellbore Parent', 'Publication date', 'Release date', 'NPDID wellbore', 'NPDID discovery', 
                    'NPDID field', 'NPDID drilling facility', 'NPDID wellbore reclassified from', 
                    'NPDID production licence drilled in', 'Date main level updated', 'Date all updated', 'Date sync NPD']

devColumnOrder = ['Well', 'Alternate 1', 'UWI number', 'Quadrant', 'Block', 'Operator', 'Licence number', 
                  'Intent', 'Intent - planned', 'Well status', 'Well content',  'Status', 'Content - planned', 'Type', 'Subsea',
                  'SPUD date', 'Completion date', 'Field', 'Predrilled entry date',
                  'Predrilled completion date', 'Drill permit', 'Discovery name',
                  'Discovery wellbore', 'KBE', 'Final vertical depth (TVD) [m RKB]',
                  'Terminal depth', 'Water depth', 'Kick off  point [m RKB]', 'Location',
                  'Country', 'Facility', 'Drilling facility type', 'Drilling facility category',
                  'Licensing activity awarded in', 'Production facility', 'Multilateral', 'Entry year', 'Completed year',
                  'Reclassified from/to wellbore', 'Plot symbol', 'Geodatum',
                  'NS degrees', 'NS minutes', 'NS seconds', 'NS code', 'EW degrees',
                  'EW minutes', 'EW seconds', 'EW code', 'Latitude', 'Longitude',
                  'Surface Y', 'Surface X', 'UTM zone',  'Grid system', 'Wellbore name part 1',
                  'Wellbore name part 2', 'Wellbore name part 3', 'Wellbore name part 4',
                  'Wellbore name part 5', 'Wellbore name part 6', 'FactPage URL',
                  'FactMaps URL', 'DISKOS Well Type', 'DISKOS Wellbore Parent',
                  'NPDID wellbore', 'NPDID discovery', 'NPDID field', 'Publication date',
                  'Release date', 'NPDID production licence drilled in',
                  'NPDID drilling facility', 'NPDID production facility',
                  'NPDID wellbore reclassified from', 'Date main level updated',
                  'Date all updated', 'Date sync NPD']
                         
#Applies the re-ordering to the dataframes
df_explo = df_explo.reindex(columns=exploColumnOrder)
df_dev = df_dev.reindex(columns=devColumnOrder)

In [35]:
df_explo.head()

,Well,Alternate 1,UWI number,Quadrant,Block,Operator,Licence number,Intent,Intent - planned,Well status,Well content,Status,Type,Subsea,SPUD date,Completion date,Field,Drill permit,Discovery name,Discovery wellbore,Bottom hole temperature [°C],Seismic line,Maximum inclination [°],KBE,Final vertical depth (TVD) [m RKB],Terminal depth,Water depth,Kick off point [m RKB],Oldest penetrated age,Oldest penetrated formation,Location,Country,Facility,Drilling facility type,Drilling facility category,Licensing activity awarded in,Multilateral,Entry year,Completed year,Reclassified from/to wellbore,Reentry activity,Plot symbol,1st level with HC formation,1st level with HC age,2nd level with HC formation,2nd level with HC age,3rd level with HC formation,3rd level with HC age,Drilling days,Reentry,Geodatum,NS degrees,NS minutes,NS seconds,NS code,EW degrees,EW minutes,EW seconds,EW code,Latitude,Longitude,Surface X,Surface Y,UTM zone,Grid system,Wellbore name part 1,Wellbore name part 2,Wellbore name part 3,Wellbore name part 4,Wellbore name part 5,Wellbore name part 6,Press Release URL,FactPage URL,FactMaps URL,DISKOS Well Type,DISKOS Wellbore Parent,Publication date,Release date,NPDID wellbore,NPDID discovery,NPDID field,NPDID drilling facility,NPDID wellbore reclassified from,NPDID production licence drilled in,Date main level updated,Date all updated,Date sync NPD
1735,7016/2-1,7016/2-1,7232,7016,2,Eni Norge AS,529,WILDCAT,WILDCAT,P & A,DRY,P & A Dry,EXPLORATION,NO,2013-07-14,2013-11-03,NaN,1465-L,NaN,NO,0.0,Survey MCD 3D troms 111 inline 1408 & trace 3478,9.1,34.0,4060.0,4061.0,1366.0,NaN,LATE PALEOCENE,SOTBAKKEN GP,BARENTS SEA,NORWAY,SCARABEO 8,SEMISUB STEEL,MOVEABLE,20,NO,2013,2013,NaN,NaN,1,NaN,NaN,NaN,NaN,,,113,NO,ED50,70,50,48.930000,N,16,33,33.230000,E,70.846925,16.559231,557093.48,7861269.63,33,ED50 / UTM zone 33N,7016,2,,1,NaN,NaN,https://www.npd.no/fakta/nyheter/resultat-av-leteboring/2013/70162-1/,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=7232,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=7232,initial,,2015-10-01,2015-10-01,7232,NaN,NaN,404415.0,0,5463559,2019-04-09,2019-04-09,21.06.2019
1736,7019/1-1,7019/1-1,4145,7019,1,Norsk Agip AS,201,WILDCAT,WILDCAT,P & A,GAS,P & A Gas,EXPLORATION,NO,2000-10-06,2000-12-03,NaN,984-L,7019/1-1,YES,108.0,94 TRM3D inline 1026 & crossline 1050,7.9,24.0,2998.0,3003.0,190.0,NaN,EARLY JURASSIC,TUBÅEN FM,BARENTS SEA,NORWAY,TRANSOCEAN ARCTIC,SEMISUB STEEL,MOVEABLE,14,NO,2000,2000,NaN,NaN,9,STØ FM,MIDDLE JURASSIC,NaN,NaN,,,59,NO,ED50,70,55,5.100000,N,19,4,22.440001,E,70.918083,19.072900,429692.26,7869589.80,34,ED50 / UTM zone 34N,7019,1,,1,NaN,NaN,,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=4145,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=4145,initial,,2003-02-11,2002-12-03,4145,1340799.0,NaN,294382.0,0,22420,2019-03-22,2019-03-22,21.06.2019
1737,7117/9-1,7117/9-1,49,7117,9,Norsk Hydro Produksjon AS,063,WILDCAT,WILDCAT,P & A,DRY,P & A Dry,EXPLORATION,NO,1982-04-20,1982-07-16,NaN,323-L,NaN,NO,112.0,79409 SP 128,7.5,23.0,3196.0,3200.0,261.0,NaN,EARLY CRETACEOUS,KOLMULE FM,BARENTS SEA,NORWAY,TREASURE SCOUT,SEMISUB STEEL,MOVEABLE,5,NO,1982,1982,NaN,NaN,1,NaN,NaN,NaN,NaN,,,88,NO,ED50,71,22,51.049999,N,17,56,5.760000,E,71.380847,17.934933,604554.55,7922625.97,33,ED50 / UTM zone 33N,7117,9,,1,NaN,NaN,,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=49,http://gis.npd.no/factmaps/html_21/?run=WellboreExpByNPDID&scale=100000&NPDID=49,initial,,2004-12-17,1984-07-16,49,NaN,NaN,287848.0,0,21300,2016-05-20,2019-03-15,21.06.2019
1738,7117/9-2,7117/9-2,13,7117,9,Norsk Hydro Produksjon AS,063,WILDCAT,WILDCAT,P & A,DRY,P & A Dry,EXPLORATION,NO,1983-05-07,1983-09-09,NaN,372-L,NaN,NO,173.0,X LINE 8103-819 SP:205+8103-610 SP:403,7.7,23.0,4991.0,5000.0,271.0,NaN,EARLY CRETACEOUS,KOLMULE FM,BARENTS SEA,

In [36]:
df_dev.head()

,Well,Alternate 1,UWI number,Quadrant,Block,Operator,Licence number,Intent,Intent - planned,Well status,Well content,Status,Content - planned,Type,Subsea,SPUD date,Completion date,Field,Predrilled entry date,Predrilled completion date,Drill permit,Discovery name,Discovery wellbore,KBE,Final vertical depth (TVD) [m RKB],Terminal depth,Water depth,Kick off point [m RKB],Location,Country,Facility,Drilling facility type,Drilling facility category,Licensing activity awarded in,Production facility,Multilateral,Entry year,Completed year,Reclassified from/to wellbore,Plot symbol,Geodatum,NS degrees,NS minutes,NS seconds,NS code,EW degrees,EW minutes,EW seconds,EW code,Latitude,Longitude,Surface Y,Surface X,UTM zone,Grid system,Wellbore name part 1,Wellbore name part 2,Wellbore name part 3,Wellbore name part 4,Wellbore name part 5,Wellbore name part 6,FactPage URL,FactMaps URL,DISKOS Well Type,DISKOS Wellbore Parent,NPDID wellbore,NPDID discovery,NPDID field,Publication date,Release date,NPDID production licence drilled in,NPDID drilling facility,NPDID production facility,NPDID wellbore reclassified from,Date main level updated,Date all updated,Date sync NPD
4962,7121/4-D-1 H,7121/4-D-1,5034,7121,4,Statoil ASA (old),099,PRODUCTION,PRODUCTION,PRODUCING,GAS,PRODUCING Gas,GAS,DEVELOPMENT,YES,2005-03-03,2005-03-28,SNØHVIT,2005-02-15,2005-02-20,2503-P,7121/4-1 Snøhvit,NO,23.0,2584.0,2680.0,337.0,NaN,BARENTS SEA,NORWAY,POLAR PIONEER,SEMISUB STEEL,MOVEABLE,8,SNØHVIT D,NO,2005,2005,NaN,90,ED50,71,35,30.790001,N,21,16,33.849998,E,71.591886,21.276069,7943649.34,509730.59,34,ED50 / UTM zone 34N,7121,4,D,1,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=5034,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=5034,initial,,5034,45038,2053062.0,NaT,2007-03-28,21596,289333.0,364278.0,0,2019-03-16,2019-03-16,24.06.2019
4963,7121/4-D-2 H,7121/4-D-2,5035,7121,4,Statoil ASA (old),099,PRODUCTION,PRODUCTION,PRODUCING,GAS,PRODUCING Gas,GAS,DEVELOPMENT,YES,2005-02-16,2005-04-14,SNØHVIT,NaT,NaT,2504-P,7121/4-1 Snøhvit,NO,23.0,2584.0,2680.0,337.0,NaN,BARENTS SEA,NORWAY,POLAR PIONEER,SEMISUB STEEL,MOVEABLE,8,SNØHVIT D,NO,2005,2005,NaN,90,ED50,71,35,31.070000,N,21,16,33.349998,E,71.591964,21.275931,7943657.99,509725.65,34,ED50 / UTM zone 34N,7121,4,D,2,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=5035,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=5035,initial,,5035,45038,2053062.0,NaT,2007-04-14,21596,289333.0,364278.0,0,2019-03-16,2019-03-16,24.06.2019
4964,7121/4-D-4 H,7121/4-D-4,5036,7121,4,Statoil ASA (old),099,PRODUCTION,PRODUCTION,PRODUCING,GAS,PRODUCING Gas,GAS,DEVELOPMENT,YES,2005-02-27,2005-05-02,SNØHVIT,2005-02-14,2005-02-18,2505-P,7121/4-1 Snøhvit,NO,23.0,2402.0,2983.0,337.0,NaN,BARENTS SEA,NORWAY,POLAR PIONEER,SEMISUB STEEL,MOVEABLE,8,SNØHVIT D,NO,2005,2005,NaN,90,ED50,71,35,30.889999,N,21,16,34.349998,E,71.591914,21.276208,7943652.46,509735.47,34,ED50 / UTM zone 34N,7121,4,D,4,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=5036,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=100000&NPDID=5036,initial,,5036,45038,2053062.0,NaT,2007-05-02,21596,289333.0,364278.0,0,2019-03-16,2019-03-16,24.06.2019
4965,7121/4-E-2 H,7121/4-E-2,5115,7121,4,Statoil ASA (old),099,PRODUCTION,PRODUCTION,PRODUCING,GAS,PRODUCING Gas,GAS,DEVELOPMENT,YES,2005-07-09,2005-07-24,SNØHVIT,2005-05-10,2005-05-17,2563-P,7121/4-1 Snøhvit,NO,23.0,2389.0,2830.0,334.0,NaN,BARENTS SEA,NORWAY,POLAR PIONEER,SEMISUB STEEL,MOVEABLE,8,SNØHVIT E,NO,2005,2005,NaN,90,ED50,71,35,41.310001,N,21,11,2.210000,E,71.594808,21.183947,7943962.92,506482.58,34,ED50 / UTM zone 34N,7121,4,E,2,NaN,NaN,http://factpages.npd.no/FactPages/Default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=5115,http://gis.npd.no/factmaps/html_21/?run=WellboreDevByNPDID&&scale=1

In [23]:
#Deletes unwanted columns (not required in IC) in Exploration and Development wells.

explo_columns_to_drop = [
    'NS degrees', 
    'NS minutes', 
    'NS seconds', 
    'NS code', 
    'EW degrees', 
    'EW minutes', 
    'EW seconds', 
    'EW code',
    'UTM zone', 
    #Rememeber: UTM zone has been superseded by 'Grid system' in IC format
    'Wellbore name part 1',
    'Wellbore name part 2', 
    'Wellbore name part 3',
    'Wellbore name part 4',
    'Wellbore name part 5',
    'Wellbore name part 6', 
    'Press Release URL',
    'FactPage URL',
    'FactMaps URL',]

dev_columns_to_drop = [
    'NS degrees', 
    'NS minutes', 
    'NS seconds', 
    'NS code', 
    'EW degrees', 
    'EW minutes', 
    'EW seconds', 
    'EW code',
    'UTM zone', 
    #Rememeber: UTM zone has been superseded by 'Grid system' in IC format
    'Wellbore name part 1',
    'Wellbore name part 2', 
    'Wellbore name part 3',
    'Wellbore name part 4',
    'Wellbore name part 5',
    'Wellbore name part 6', 
    'FactPage URL',
    'FactMaps URL',]

#Deletes the columns from the dataframes.
df_explo.drop(explo_columns_to_drop, axis=1, inplace=True)
df_dev.drop(dev_columns_to_drop, axis=1, inplace=True)
df_dev.head()

,Well,Alternate 1,UWI number,Quadrant,Block,Operator,Licence number,Intent,Intent - planned,Well status,Well content,Status,Content - planned,Type,Subsea,SPUD date,Completion date,Field,Predrilled entry date,Predrilled completion date,Drill permit,Discovery name,Discovery wellbore,KBE,Final vertical depth (TVD) [m RKB],Terminal depth,Water depth,Kick off point [m RKB],Location,Country,Facility,Drilling facility type,Drilling facility category,Licensing activity awarded in,Production facility,Multilateral,Entry year,Completed year,Reclassified from/to wellbore,Plot symbol,Geodatum,Latitude,Longitude,Surface Y,Surface X,Grid system,DISKOS Well Type,DISKOS Wellbore Parent,NPDID wellbore,NPDID discovery,NPDID field,Publication date,Release date,NPDID production licence drilled in,NPDID drilling facility,NPDID production facility,NPDID wellbore reclassified from,Date main level updated,Date all updated,Date sync NPD
4962,7121/4-D-1 H,7121/4-D-1,5034,7121,4,Statoil ASA (old),099,PRODUCTION,PRODUCTION,PRODUCING,GAS,PRODUCING Gas,GAS,DEVELOPMENT,YES,2005-03-03,2005-03-28,SNØHVIT,2005-02-15,2005-02-20,2503-P,7121/4-1 Snøhvit,NO,23.0,2584.0,2680.0,337.0,NaN,BARENTS SEA,NORWAY,POLAR PIONEER,SEMISUB STEEL,MOVEABLE,8,SNØHVIT D,NO,2005,2005,NaN,90,ED50,71.591886,21.276069,7943649.34,509730.59,ED50 / UTM zone 34N,initial,,5034,45038,2053062.0,NaT,2007-03-28,21596,289333.0,364278.0,0,2019-03-16,2019-03-16,24.06.2019
4963,7121/4-D-2 H,7121/4-D-2,5035,7121,4,Statoil ASA (old),099,PRODUCTION,PRODUCTION,PRODUCING,GAS,PRODUCING Gas,GAS,DEVELOPMENT,YES,2005-02-16,2005-04-14,SNØHVIT,NaT,NaT,2504-P,7121/4-1 Snøhvit,NO,23.0,2584.0,2680.0,337.0,NaN,BARENTS SEA,NORWAY,POLAR PIONEER,SEMISUB STEEL,MOVEABLE,8,SNØHVIT D,NO,2005,2005,NaN,90,ED50,71.591964,21.275931,7943657.99,509725.65,ED50 / UTM zone 34N,initial,,5035,45038,2053062.0,NaT,2007-04-14,21596,289333.0,364278.0,0,2019-03-16,2019-03-16,24.06.2019
4964,7121/4-D-4 H,7121/4-D-4,5036,7121,4,Statoil ASA (old),099,PRODUCTION,PRODUCTION,PRODUCING,GAS,PRODUCING Gas,GAS,DEVELOPMENT,YES,2005-02-27,2005-05-02,SNØHVIT,2005-02-14,2005-02-18,2505-P,7121/4-1 Snøhvit,NO,23.0,2402.0,2983.0,337.0,NaN,BARENTS SEA,NORWAY,POLAR PIONEER,SEMISUB STEEL,MOVEABLE,8,SNØHVIT D,NO,2005,2005,NaN,90,ED50,71.591914,21.276208,7943652.46,509735.47,ED50 / UTM zone 34N,initial,,5036,45038,2053062.0,NaT,2007-05-02,21596,289333.0,364278.0,0,2019-03-16,2019-03-16,24.06.2019
4965,7121/4-E-2 H,7121/4-E-2,5115,7121,4,Statoil ASA (old),099,PRODUCTION,PRODUCTION,PRODUCING,GAS,PRODUCING Gas,GAS,DEVELOPMENT,YES,2005-07-09,2005-07-24,SNØHVIT,2005-05-10,2005-05-17,2563-P,7121/4-1 Snøhvit,NO,23.0,2389.0,2830.0,334.0,NaN,BARENTS SEA,NORWAY,POLAR PIONEER,SEMISUB STEEL,MOVEABLE,8,SNØHVIT E,NO,2005,2005,NaN,90,ED50,71.594808,21.183947,7943962.92,506482.58,ED50 / UTM zone 34N,initial,,5115,45038,2053062.0,NaT,2007-07-24,21596,289333.0,364315.0,0,2019-03-16,2019-03-16,24.06.2019
4966,7121/4-E-3 H,7121/4-E-3,5116,7121,4,Statoil ASA (old),099,PRODUCTION,PRODUCTION,PRODUCING,GAS,PRODUCING Gas,GAS,DEVELOPMENT,YES,2005-05-25,2005-07-06,SNØHVIT,2005-05-11,2005-05-14,2564-P,7121/4-1 Snøhvit,NO,23.0,2406.0,2708.0,334.0,NaN,BARENTS SEA,NORWAY,POLAR PIONEER,SEMISUB STEEL,MOVEABLE,8,SNØHVIT E,NO,2005,2005,NaN,90,ED50,71.594836,21.184086,7943966.04,506487.47,ED50 / UTM zone 34N,initial,,5116,45038,2053062.0,NaT,2007-07-06,21596,289333.0,364315.0,0,2019-03-16,2019-03-16,24.06.2019


In [44]:
#Final quality-check: prints out all unique values for selected columns.

def lstheaderfields (*args):
    for arg in args:
        print(arg, '---')
        print('')
        words = [x for x in df_explo[arg].unique()]
        print('Exploration wells:')
        print(words)
        print('')
        words = [x for x in df_dev[arg].unique()]
        print('Development wells:')
        print(words)
        print("")
            
lstheaderfields('Operator', 'Status', 'Type', 'Field', 'Intent', 'Intent - planned', 'Geodatum', 'Grid system')

#Do the same but for one column at a time (prints without any brackets, quotes or commas)
# lstoperator = sorted(set(df_explo['Operator'].astype(str)))
# print('Operator:', ', '.join(lstoperator))
# print('')

#ENSURE UTM ZONES ARE INSTALLED IN YOUR IC PROJECT USING PROJECT PROPERTIES > COORDS TAB.

Operator ---

Exploration wells:
['Eni Norge AS', 'Norsk Agip AS', 'Norsk Hydro Produksjon AS', 'Elf Aquitaine Norge A/S', 'Den norske stats oljeselskap a.s', 'Statoil Petroleum AS', 'A/S Norske Shell', 'Lundin Norway AS', 'Statoil ASA (old)', 'StatoilHydro ASA', 'Esso Exploration and Production Norway A/S', 'DONG E&P Norge AS', 'Total Norge AS', 'StatoilHydro Petroleum AS', 'Saga Petroleum ASA', 'GDF SUEZ E&P Norge AS', 'Norsk Hydro Petroleum AS', 'Conoco Norway Inc.', 'Equinor Energy AS', 'Repsol Exploration Norge AS', 'Aker BP ASA', 'Det norske oljeselskap ASA', 'Wintershall Norge AS', 'Total E&P Norge AS', 'Norwegian Energy Company ASA', 'Mobil Development Norway AS', 'DEA Norge AS', 'Mobil Exploration Norway INC', 'Spirit Energy Norge AS', 'OMV (Norge) AS']

Development wells:
['Statoil ASA (old)', 'Statoil Petroleum AS', 'Equinor Energy AS', 'Eni Norge AS']

Status ---

Exploration wells:
['P & A Dry', 'P & A Gas', 'P & A Oil Shows', 'P & A Shows', 'P & A Gas/Condensate', 'SUSPEN

In [ ]:
#Outputs two CSV files for Well Headers, one for Exploration and one for Development.

df_explo.to_csv('IC_wellbore_exploration.csv', encoding='UTF-8', index=False)
df_dev.to_csv('IC_wellbore_development.csv', encoding='UTF-8', index=False)

#with open('wellbore_exploration.csv', 'w', encoding='UTF-8', newline='') as csvarchive:
#    writer = csv.writer(csvarchive)